In [1]:
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

import boto3
import pandas as pd

sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'predictive-maintainence-data' # Mention the created S3 bucket name here
print("Using bucket " + bucket)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\vishw\AppData\Local\sagemaker\sagemaker\config.yaml
Using bucket predictive-maintainence-data


# DATA PREPROCESSING

In [2]:
raw_data = pd.read_csv('predictive_maintenance.csv')
df = raw_data.copy()
df.head()

UDI Product ID Type  Air temperature [K]  Process temperature [K]  \
0    1     M14860    M                298.1                    308.6   
1    2     L47181    L                298.2                    308.7   
2    3     L47182    L                298.1                    308.5   
3    4     L47183    L                298.2                    308.6   
4    5     L47184    L                298.2                    308.7   

   Rotational speed [rpm]  Torque [Nm]  Tool wear [min]  Target Failure Type  
0                    1551         42.8                0       0   No Failure  
1                    1408         46.3                3       0   No Failure  
2                    1498         49.4                5       0   No Failure  
3                    1433         39.5                7       0   No Failure  
4                    1408         40.0                9       0   No Failure

In [3]:
# Dropping 'UDI' and 'Product ID' from the dataset
df.drop(['UDI', 'Product ID'], axis=1, inplace=True)

# Let's change the name of the columns.
df = df.rename(mapper={'Air temperature [K]': 'Air temperature',
                       'Process temperature [K]': 'Process temperature',
                       'Rotational speed [rpm]': 'Rotational speed',
                       'Torque [Nm]': 'Torque', 
                       'Tool wear [min]': 'Tool wear'}, axis=1)



### Ornidal encoding

In [4]:
columns_failure_type = df['Failure Type'].unique().tolist()


# Pass on the categorical values
ord_enc = OrdinalEncoder(categories=[['L', 'M', 'H'], columns_failure_type])

# Fit and transform the data for feature 'Type' and target 'Failure Type'
new_dat = ord_enc.fit_transform(df[['Type', 'Failure Type']])
new_dat
df.drop(['Type', 'Failure Type'], axis=1, inplace=True)

new_df = pd.DataFrame(new_dat, 
                      index=df.index, 
                      columns=['Type', 'Failure Type'])

df = pd.concat([df, new_df], axis=1)

### Scaling data

Since 'Rotational spede' and 'Torque' have some outliers, I am going to use RobustScaler:

In [5]:
df_scaled = df.copy()

# Scaling Rotational speed and torque with robust scaler:
columns = ['Rotational speed', 'Torque']
scaler = RobustScaler()
features_scaled = scaler.fit_transform(df[columns])
features_scaled = pd.DataFrame(features_scaled, columns=columns)
df_scaled.drop(columns, axis=1, inplace=True)
df_scaled = pd.concat([df_scaled,features_scaled], axis=1)

df_scaled.head(5)

Air temperature  Process temperature  Tool wear  Target  Type  \
0            298.1                308.6          0       0   1.0   
1            298.2                308.7          3       0   0.0   
2            298.1                308.5          5       0   0.0   
3            298.2                308.6          7       0   0.0   
4            298.2                308.7          9       0   0.0   

   Failure Type  Rotational speed    Torque  
0           0.0          0.253968  0.198529  
1           0.0         -0.502646  0.455882  
2           0.0         -0.026455  0.683824  
3           0.0         -0.370370 -0.044118  
4           0.0         -0.502646 -0.007353

The rest of the features will be scaled with MinMax scaling:

In [6]:
columns = ['Air temperature', 'Process temperature', 'Tool wear']
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(df[columns])
features_scaled = pd.DataFrame(features_scaled, columns=columns)
df_scaled.drop(columns, axis=1, inplace=True)
df_scaled = pd.concat([df_scaled, features_scaled], axis=1)

df_scaled.head()

Target  Type  Failure Type  Rotational speed    Torque  Air temperature  \
0       0   1.0           0.0          0.253968  0.198529         0.304348   
1       0   0.0           0.0         -0.502646  0.455882         0.315217   
2       0   0.0           0.0         -0.026455  0.683824         0.304348   
3       0   0.0           0.0         -0.370370 -0.044118         0.315217   
4       0   0.0           0.0         -0.502646 -0.007353         0.315217   

   Process temperature  Tool wear  
0             0.358025   0.000000  
1             0.370370   0.011858  
2             0.345679   0.019763  
3             0.358025   0.027668  
4             0.370370   0.035573

In [7]:
import os
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd

# Assuming df_scaled is your preprocessed dataframe
# Splitting the data
X = df_scaled.drop(['Target', 'Failure Type'], axis=1)
y = df_scaled['Target']

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)

for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]

# Combine X and y for train and test sets
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Directory to save CSV files
csv_dir = 'train_test_data'  # Use a relative path

# Create directory if it does not exist
os.makedirs(csv_dir, exist_ok=True)

# Save train and test data to CSV files
train_csv = os.path.join(csv_dir, 'train.csv')
test_csv = os.path.join(csv_dir, 'test.csv')
train_df.to_csv(train_csv, index=False)
test_df.to_csv(test_csv, index=False)

print(f"Train CSV file saved at: {train_csv}")
print(f"Test CSV file saved at: {test_csv}")


Train CSV file saved at: train_test_data\train.csv
Test CSV file saved at: train_test_data\test.csv


In [8]:
print("Current Working Directory:", os.getcwd())


Current Working Directory: C:\Users\vishw\Documents\Prudhvi_krishna_capstone


## Uploading csv files to s3

In [9]:
# Upload CSV files to S3
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
bucket = 'predictive-maintainence-data'  # Mention the created S3 bucket name here
sk_prefix = "sagemakertraining"

# Upload train data
trainpath = sess.upload_data(
    path=train_csv, bucket=bucket, key_prefix=sk_prefix
)

# Upload test data
testpath = sess.upload_data(
    path=test_csv, bucket=bucket, key_prefix=sk_prefix
)

print("Train data path in S3:", trainpath)
print("Test data path in S3:", testpath)

# Checking the stratified split
print('Checking the stratified split...')
print('Target proportion in original dataset:')
print(df_scaled['Target'].value_counts(normalize=True))

print('Target proportion in y_train dataset:')
print(y_train.value_counts(normalize=True))

print('Target proportion in y_test dataset:')
print(y_test.value_counts(normalize=True))

Train data path in S3: s3://predictive-maintainence-data/sagemakertraining/train.csv
Test data path in S3: s3://predictive-maintainence-data/sagemakertraining/test.csv
Checking the stratified split...
Target proportion in original dataset:
Target
0    0.9661
1    0.0339
Name: proportion, dtype: float64
Target proportion in y_train dataset:
Target
0    0.966133
1    0.033867
Name: proportion, dtype: float64
Target proportion in y_test dataset:
Target
0    0.966
1    0.034
Name: proportion, dtype: float64


In [10]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.model_selection import StratifiedShuffleSplit, cross_validate, cross_val_score
from imblearn.ensemble import BalancedRandomForestClassifier
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO 
import argparse
import os
import numpy as np
import pandas as pd
    
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf
    
if __name__ == "__main__":

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train.csv")
    parser.add_argument("--test-file", type=str, default="test.csv")

    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    
    features = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    
    print("Label column is: ",label)
    
    print("Data Shape: ")
    print("---- SHAPE OF TRAINING DATA ----")
    print(X_train.shape)
    print(y_train.shape)
    print("---- SHAPE OF TESTING DATA ----")
    print(X_test.shape)
    print(y_test.shape)

    print("Training BalancedRandomForestClassifier Model.....")
    model = BalancedRandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, n_jobs=-1)
    model.fit(X_train, y_train)

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)
    print("Model persisted at " + model_path)

    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred_test)
    test_rep = classification_report(y_test, y_pred_test)

    # Cross validation metrics test data
    cv = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=42)
    scoring = ["f1_macro", "precision_macro", "recall_macro"]
    cross_val_scores = cross_validate(model, X_train, y_train, scoring=scoring)
    roc_auc_test_cv = round(cross_val_score(model, X_train, y_train, cv=cv, n_jobs=-1, scoring="roc_auc").mean(), 4)
    f1_test_cv = round(cross_val_scores["test_f1_macro"].mean(), 4)
    precision_test_cv = round(cross_val_scores["test_precision_macro"].mean(), 4)
    recall_test_cv = round(cross_val_scores["test_recall_macro"].mean(), 4)

    # Print metrics
    print(f'Cross val roc auc test data: {roc_auc_test_cv}')
    print(f'Cross val macro f1 score test data: {f1_test_cv}')
    print(f'Cross val macro precision score test data: {precision_test_cv}')
    print(f'Cross val macro recall score test data: {recall_test_cv}')

    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)


Overwriting script.py


In [11]:
import sagemaker
from sagemaker.sklearn import SKLearn
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

role = 'arn:aws:iam::381492047582:role/service-role/AmazonSageMaker-ExecutionRole-20240716T085853'
bucket = 'predictive-maintainence-data'
train_prefix = 'sagemakertraining'
script_prefix = 'training_script'
output_prefix = 'output'

# S3 path to the script tar.gz
script_s3_path = f's3://{bucket}/{script_prefix}/script.tar.gz'
output_path = f's3://{bucket}/{output_prefix}/'

logger.info("Creating SKLearn estimator")
try:
    sklearn = SKLearn(
        entry_point='script.py',
        role=role,
        instance_type='ml.m5.xlarge',
        framework_version='0.23-1',
        instance_count=1,
        hyperparameters={
            'n_estimators': 100,
            'random_state': 42
        },
        output_path=output_path,
        code_location=script_s3_path,
        dependencies=['requirements.txt'],
        enable_sagemaker_metrics=True,
        debugger_hook_config=False,
        metric_definitions=[
            {"Name": "validation:accuracy", "Regex": "validation-accuracy=([0-9\\.]+)"},
            {"Name": "train:accuracy", "Regex": "train-accuracy=([0-9\\.]+)"}
        ]
    )
    logger.info("SKLearn estimator created successfully")
except Exception as e:
    logger.error("Error creating SKLearn estimator: %s", e)
    raise

logger.info("Setting up training and test inputs")
try:
    train_input = sagemaker.inputs.TrainingInput(f's3://{bucket}/{train_prefix}/train.csv', content_type='text/csv')
    test_input = sagemaker.inputs.TrainingInput(f's3://{bucket}/{train_prefix}/test.csv', content_type='text/csv')
    logger.info("Training and test inputs set up successfully")
except Exception as e:
    logger.error("Error setting up training and test inputs: %s", e)
    raise



INFO:__main__:Creating SKLearn estimator
INFO:__main__:SKLearn estimator created successfully
INFO:__main__:Setting up training and test inputs
INFO:__main__:Training and test inputs set up successfully


In [12]:
logger.info("Submitting the training job")
try:
    sklearn.fit({'train': train_input, 'test': test_input})
    logger.info("Training job submitted successfully")
except Exception as e:
    logger.error("Error submitting training job: %s", e)
    raise


INFO:__main__:Submitting the training job
INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2024-07-23-16-05-31-761


2024-07-23 16:05:32 Starting - Starting the training job....

KeyboardInterrupt: 

# Deployment

In [ ]:
import sagemaker
from sagemaker.sklearn.model import SKLearnModel

# Replace 'your-role-arn' with the actual ARN of the role you created
role = 'arn:aws:iam::381492047582:role/service-role/AmazonSageMaker-ExecutionRole-20240716T085853'

model = SKLearnModel(
    model_data='s3://predictive-maintainence-data/model/model.tar.gz',
    role=role,
    entry_point='inference.py',  # This script will define how the model is used for inference
    framework_version='0.23-1',
    py_version='py3'
)

from time import gmtime, strftime

endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name
)
